### Load tensorflow

In [0]:
import tensorflow as tf

In [8]:
tf.__version__

'1.14.0'

### Enable Eager Execution if you are using tensorflow 1.x

In [0]:
##Enable Eager Execution
tf.enable_eager_execution()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [49]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns = ['date','symbol'])

In [51]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data_new = data.head(1000)

### Convert Float64 to Float32

In [0]:
data_new = data_new.astype('float32')


In [0]:
X = data_new.drop(columns='close')
Y = data_new.close

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size =0.2,random_state = 123)

In [69]:
X_train.shape

(800, 4)

### Normalize Train and Test Data 

In [0]:
from sklearn.preprocessing import Normalizer

In [62]:
train_x

array([[2.2876651e-05, 2.2689075e-05, 2.3034214e-05, 1.0000000e+00],
       [4.7349291e-05, 4.6249999e-05, 4.7349291e-05, 1.0000000e+00],
       [7.6225746e-07, 7.5391358e-07, 7.7544627e-07, 1.0000000e+00],
       ...,
       [2.3780945e-05, 2.3505878e-05, 2.4289406e-05, 1.0000000e+00],
       [1.7972880e-05, 1.7853366e-05, 1.8119972e-05, 1.0000000e+00],
       [3.1503550e-05, 3.1358217e-05, 3.1951997e-05, 1.0000000e+00]],
      dtype=float32)

In [70]:
y_train

512     30.350000
685     52.730000
997     28.770000
927     14.940000
376     26.790001
788     23.209997
91     123.910004
170    128.550003
584     25.000000
90     122.970001
75     123.790001
747     69.430000
969     37.380001
368     56.060001
893     98.089996
529     27.430000
870     25.680000
97     126.620003
897     66.949997
272     22.330000
881     10.960000
936     37.169998
270     29.889999
708     18.719999
483     23.340000
610     27.120001
431     59.230000
754     37.700001
729     66.019997
279     38.599998
          ...    
208    125.900002
608     24.629999
420     49.430000
253     40.380001
846     47.560001
339     30.500000
409     48.880001
111    124.959999
224    124.820000
942     30.350000
544     30.950001
73     122.309998
47     120.629997
638     20.440001
113    124.839996
96     125.949997
737     29.330000
214    118.790001
569     45.579998
123    124.309998
106    129.250000
595     60.599998
17     114.470001
742     36.779999
98     126

In [0]:
transformer = Normalizer()
train_x = transformer.fit_transform(X_train)
test_x = transformer.fit_transform(X_test)

## Building the graph in tensorflow

2.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

3.Prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

4.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b 

In [0]:
import numpy as np

## Execute the Graph for 100 epochs and observe the loss

In [73]:
for i in range(100):
    
    w, b = train(train_x, np.array(y_train), w, b)
    print('Current Loss on iteration', i, loss(np.array(y_train), prediction(train_x, w, b)).numpy())

Current Loss on iteration 0 7968.088
Current Loss on iteration 1 7639.777
Current Loss on iteration 2 7337.2095
Current Loss on iteration 3 7058.362
Current Loss on iteration 4 6801.3794
Current Loss on iteration 5 6564.5425
Current Loss on iteration 6 6346.2725
Current Loss on iteration 7 6145.112
Current Loss on iteration 8 5959.727
Current Loss on iteration 9 5788.877
Current Loss on iteration 10 5631.4185
Current Loss on iteration 11 5486.3057
Current Loss on iteration 12 5352.5703
Current Loss on iteration 13 5229.322
Current Loss on iteration 14 5115.733
Current Loss on iteration 15 5011.049
Current Loss on iteration 16 4914.574
Current Loss on iteration 17 4825.6616
Current Loss on iteration 18 4743.72
Current Loss on iteration 19 4668.2026
Current Loss on iteration 20 4598.608
Current Loss on iteration 21 4534.4663
Current Loss on iteration 22 4475.356
Current Loss on iteration 23 4420.877
Current Loss on iteration 24 4370.669
Current Loss on iteration 25 4324.4014
Current Loss

### Get the shapes and values of W and b

In [74]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[2.5760543e-03]
 [2.5527244e-03]
 [2.5969257e-03]
 [3.3135368e+01]]
Bias:
 [33.13537]


In [75]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

prediction(train_x,w,b)

<tf.Tensor: id=5131535, shape=(800, 1), dtype=float32, numpy=
array([[66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27074 ],
       [66.27072 ],
       [66.27074 ],
       [66.27074 ]

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
#Initialize Sequential Graph (model)
model = tf.keras.Sequential()

#Add Dense layer for prediction - Keras declares weights and bias automatically
model.add(tf.keras.layers.Dense(1, input_shape=(4,)))

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer='sgd', loss='mse')

### Execute the model

In [82]:
model.fit(train_x, y_train, epochs=100)

Epoch 1/100
800/800 [==============================] - 0s 424us/sample - loss: 5866.3665
Epoch 2/100
800/800 [==============================] - 0s 29us/sample - loss: 4056.8660
Epoch 3/100
800/800 [==============================] - 0s 30us/sample - loss: 3817.3034
Epoch 4/100
800/800 [==============================] - 0s 31us/sample - loss: 3787.5310
Epoch 5/100
800/800 [==============================] - 0s 29us/sample - loss: 3785.6210
Epoch 6/100
800/800 [==============================] - 0s 32us/sample - loss: 3785.5769
Epoch 7/100
800/800 [==============================] - 0s 29us/sample - loss: 3785.0240
Epoch 8/100
800/800 [==============================] - 0s 32us/sample - loss: 3782.1934
Epoch 9/100
800/800 [==============================] - 0s 36us/sample - loss: 3783.6998
Epoch 10/100
800/800 [==============================] - 0s 36us/sample - loss: 3783.6302
Epoch 11/100
800/800 [==============================] - 0s 34us/sample - loss: 3784.6252
Epoch 12/100
800/800 [=======

In [83]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.get_weights()

[array([[-0.25881302],
        [ 0.04899972],
        [ 0.07619543],
        [33.563053  ]], dtype=float32), array([34.632206], dtype=float32)]

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('Iris-2.csv')

### Splitting the data into feature set and target set

In [87]:
iris.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [0]:
X=iris.drop(columns=['Id','Species'])
Y=iris.Species

In [0]:
#X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size =0.3,random_state = 123)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [116]:
Y.value_counts()

Iris-setosa        50
Iris-virginica     50
Iris-versicolor    50
Name: Species, dtype: int64

In [0]:
Y = pd.get_dummies(Y)
#testX = tf.keras.utils.to_categorical(np.array(X_test), num_classes=4)
#trainY = tf.keras.utils.to_categorical(y_train, num_classes=4)
#testY = tf.keras.utils.to_categorical(y_test, num_classes=4)

### Divide the dataset into Training and test (70:30)

In [0]:
X_train,X_test,y_train,y_test= train_test_split(X,Y,test_size =0.3,random_state = 123)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Re,input_shape=(4,)))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape=(4,)))

#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='sigmoid'))
#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
#Compile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Fitting the model and predicting 

In [132]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=80,
          batch_size=10)

Train on 105 samples, validate on 45 samples
Epoch 1/80
105/105 [==============================] - 0s 274us/sample - loss: 1.0521 - acc: 0.4000 - val_loss: 1.0644 - val_acc: 0.2667
Epoch 2/80
105/105 [==============================] - 0s 233us/sample - loss: 1.0458 - acc: 0.4381 - val_loss: 1.0622 - val_acc: 0.2667
Epoch 3/80
105/105 [==============================] - 0s 239us/sample - loss: 1.0453 - acc: 0.4095 - val_loss: 1.0610 - val_acc: 0.2667
Epoch 4/80
105/105 [==============================] - 0s 222us/sample - loss: 1.0430 - acc: 0.4286 - val_loss: 1.0601 - val_acc: 0.2667
Epoch 5/80
105/105 [==============================] - 0s 237us/sample - loss: 1.0472 - acc: 0.4476 - val_loss: 1.0583 - val_acc: 0.2667
Epoch 6/80
105/105 [==============================] - 0s 208us/sample - loss: 1.0421 - acc: 0.4381 - val_loss: 1.0564 - val_acc: 0.2667
Epoch 7/80
105/105 [==============================] - 0s 236us/sample - loss: 1.0409 - acc: 0.4286 - val_loss: 1.0554 - val_acc: 0.2667
Epo

### Report Accuracy of the predicted values

In [129]:
pred = model.predict(X_test[:])
pred

array([[0.28994578, 0.39147985, 0.3185744 ],
       [0.29842725, 0.37391034, 0.32766235],
       [0.29632148, 0.37498888, 0.32868966],
       [0.29410332, 0.38428614, 0.32161048],
       [0.2845145 , 0.4137634 , 0.30172214],
       [0.29680473, 0.37659574, 0.32659954],
       [0.2860022 , 0.40128824, 0.3127096 ],
       [0.2864384 , 0.40860346, 0.30495805],
       [0.28206563, 0.41515535, 0.30277905],
       [0.28935093, 0.39093882, 0.31971028],
       [0.2986132 , 0.37529546, 0.32609132],
       [0.30007333, 0.38145933, 0.31846738],
       [0.28986698, 0.38993487, 0.32019815],
       [0.29841793, 0.37333077, 0.32825136],
       [0.29897013, 0.3741832 , 0.32684666],
       [0.2994279 , 0.36857226, 0.3319999 ],
       [0.29003954, 0.40583637, 0.30412406],
       [0.2824863 , 0.41584933, 0.30166444],
       [0.29448703, 0.383553  , 0.32195994],
       [0.28326896, 0.415369  , 0.30136204],
       [0.28893968, 0.40689757, 0.3041627 ],
       [0.29436412, 0.38260844, 0.32302746],
       [0.

In [135]:
pred[3]

array([0.29410332, 0.38428614, 0.32161048], dtype=float32)

In [134]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 4)                 16        
_________________________________________________________________
dense_12 (Dense)             (None, 10)                50        
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 27        
Total params: 181
Trainable params: 173
Non-trainable params: 8
_________________________________________________________________


In [0]:
model.sc